<a href="https://colab.research.google.com/github/blockmoo/Python/blob/main/DACON_%EC%A3%BC%EC%B0%A8%EC%88%98%EC%9A%94%EC%98%88%EC%B8%A1_%ED%8C%80%ED%94%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **데이콘**
[배경]

아파트 단지 내 필요한 주차대수는 ①법정주차대수 ②장래주차수요 중 큰 값에 따라 결정하게되어 있어, 정확한 ②장래주차수요의 산정을 필요로 합니다.  

현재 ②장래주차수요는 ‘주차원단위’와 ‘건축연면적’을 기초로하여 산출되고 있으며, ‘주차원단위’는 신규 건축예정 부지 인근의 유사 단지를 피크 시간대 방문하여 주차된 차량대수를 세는 방법으로 조사하고 있습니다.  

이 경우 인력조사로 인한 오차발생, 현장조사 시점과 실제 건축시점과의 시간차 등의 문제로 과대 또는 과소 산정의 가능성을 배제할 수 없습니다.  


[주제]



🏠 유형별 임대주택 설계 시 단지 내 적정 🅿️ 주차 수요를 예측

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/데이콘 주차수요 예측 AI 경진대회")
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/데이콘 주차수요 예측 AI 경진대회'

In [2]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import font_manager, rc
plt.rc('font', family='NanumBarunGothic') 

In [3]:
age = pd.read_csv("age_gender_info.csv")
age.head(3)

,지역,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,경상북도,0.030158,0.033195,0.056346,0.061360,0.060096,0.067859,0.053433,0.049572,0.083660,0.072613,0.087149,0.072146,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
1,경상남도,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,대전광역시,0.028197,0.029092,0.040490,0.042793,0.060834,0.064247,0.068654,0.066848,0.074667,0.067925,0.085751,0.068819,0.088468,0.070261,0.051010,0.037143,0.032455,0.013751,0.006494,0.001740,0.000298,0.000066


In [4]:
# 불필요한 데이터 제거 파일 : 데이터 공지 참고
train2 = pd.read_csv("train2.csv", encoding='cp949')
test2 = pd.read_csv('test2.csv', encoding='cp949')
submission = pd.read_csv('sample_submission.csv')
train2.head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17,A,9216000,82940,0.0,3,624,205
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17,A,12672000,107130,0.0,3,624,205
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17,A,12672000,107130,0.0,3,624,205


In [5]:
# 지하철역 수, 버스정류장 수 컬럼명을 지하철, 버스로 변경
print(train2.columns)
train2.rename(columns = {'도보 10분거리 내 지하철역 수(환승노선 수 반영)' : '지하철',"도보 10분거리 내 버스정류장 수" : "버스"}, inplace = True)
test2.rename(columns = {'도보 10분거리 내 지하철역 수(환승노선 수 반영)' : '지하철',"도보 10분거리 내 버스정류장 수" : "버스"}, inplace = True)
print(train2.columns)

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')
Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스', '단지내주차면수', '등록차량수'],
      dtype='object')


In [6]:
# 지하철 수 NULL값 처리: 모두 0 (직접 실제 아파트와 비교)
train2.지하철 = train2.지하철.fillna(0)
train2.tail(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2866,C2532,239,아파트,강원도,국민임대,51.73,34,7,A,14005000,142310,0.0,1,166,146
2867,C2532,239,아파트,강원도,국민임대,51.96,114,7,A,14005000,142310,0.0,1,166,146
2868,C2532,239,아파트,강원도,국민임대,54.95,19,7,A,14830000,151030,0.0,1,166,146


In [7]:
uni_all = train2.단지코드.unique()  #unique한 단지코드만 추출
len(uni_all)

411

In [8]:
#실질세대수 변수 추가(총 세대수 삭제 안함, 추후 불필요할 시 삭제)
train2 = train2.set_index('단지코드')
train2['ssd'] = train2.groupby('단지코드')['전용면적별세대수'].sum()
train2['dif'] = train2.총세대수 - train2.ssd
train2 = train2.reset_index()
train2.tail(3) 

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수,ssd,dif
2866,C2532,239,아파트,강원도,국민임대,51.73,34,7,A,14005000,142310,0.0,1,166,146,239,0
2867,C2532,239,아파트,강원도,국민임대,51.96,114,7,A,14005000,142310,0.0,1,166,146,239,0
2868,C2532,239,아파트,강원도,국민임대,54.95,19,7,A,14830000,151030,0.0,1,166,146,239,0


In [9]:
# 총세대 수와 실질세대수가 다른 단지 확인 
# 실질세대수 차이
train2.groupby(train2.dif).단지코드.unique()

dif
0      [C2515, C1407, C1945, C1470, C1898, C1244, C11...
1      [C1925, C1312, C2013, C1424, C2319, C1068, C26...
2                           [C2568, C2245, C2549, C1732]
3                    [C1850, C1695, C2556, C2298, C2433]
4                                                [C1910]
5                                         [C2139, C2508]
6                                         [C2225, C1156]
7                                                [C1584]
8                                         [C2142, C2186]
9                                                [C2362]
10                                               [C2520]
14                                               [C1970]
94                                               [C1490]
123                                              [C2620]
124                                              [C1344]
203                                              [C1024]
230                                              [C2470]
267                        

In [10]:
# test2도 똑같은 프로세스
# 지하철 수 NULL값 처리: 모두 0 (직접 실제 아파트와 비교)
test2.지하철 = test2.지하철.fillna(0)
test2.tail(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
1005,C2189,382,아파트,전라북도,국민임대,29.19,20,45,H,6872000,106400,0.0,2,300
1006,C2189,382,아파트,전라북도,국민임대,39.45,202,45,H,13410000,144600,0.0,2,300
1007,C2189,382,아파트,전라북도,국민임대,46.23,60,45,H,18689000,166500,0.0,2,300


In [11]:
uni_all = test2.단지코드.unique()  #unique한 단지코드만 추출
len(uni_all)

147

In [12]:
#실질세대수 변수 추가(총 세대수 삭제 안함, 추후 불필요할 시 삭제)
test2 = test2.set_index('단지코드')
test2['ssd'] = test2.groupby('단지코드')['전용면적별세대수'].sum()
test2['dif'] = test2.총세대수 - test2.ssd
test2 = test2.reset_index()
test2.tail(3) 

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,ssd,dif
1005,C2189,382,아파트,전라북도,국민임대,29.19,20,45,H,6872000,106400,0.0,2,300,378,4
1006,C2189,382,아파트,전라북도,국민임대,39.45,202,45,H,13410000,144600,0.0,2,300,378,4
1007,C2189,382,아파트,전라북도,국민임대,46.23,60,45,H,18689000,166500,0.0,2,300,378,4


In [13]:
# 총세대 수와 실질세대수가 다른 단지 확인 
# 실질세대수 차이
test2.groupby(test2.dif).단지코드.unique()

dif
0      [C1072, C1128, C1456, C1840, C1332, C1563, C17...
1                                                [C2411]
2                                                [C1683]
3                                  [C1229, C1414, C2456]
4                                         [C1038, C2189]
5                           [C1030, C1349, C1266, C1267]
8                                  [C1749, C2363, C2174]
9                                                [C2043]
12                                               [C1812]
14                                               [C2404]
122                                              [C2497]
Name: 단지코드, dtype: object

In [14]:
# 임대가 있는 아파트 단지 찾기
임대상가 = train2.loc[train2.공급유형 == "임대상가"]
임대단지코드 = 임대상가.단지코드.unique()
임대상가아파트 = train2.loc[train2.단지코드.isin(임대단지코드)]
임대상가아파트.head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수,ssd,dif
78,C1925,601,아파트,강원도,영구임대,26.37,298,9,C,8052000,130850,0.0,4,117,75,600,1
79,C1925,601,아파트,강원도,영구임대,31.32,298,9,C,9174000,144460,0.0,4,117,75,600,1
80,C1925,601,상가,강원도,임대상가,32.10,1,9,D,NaN,NaN,0.0,4,117,75,600,1


In [15]:
# 임대상가가 있는 아파트 유형 확인
임대상가아파트.groupby(임대상가아파트.단지코드).공급유형.unique() # 33개 단지코드

단지코드
C1004               [영구임대, 임대상가]
C1056               [영구임대, 임대상가]
C1109               [영구임대, 임대상가]
C1206               [영구임대, 임대상가]
C1439               [영구임대, 임대상가]
C1476               [영구임대, 임대상가]
C1616               [영구임대, 임대상가]
C1698               [영구임대, 임대상가]
C1704               [영구임대, 임대상가]
C1722    [공공임대(50년), 영구임대, 임대상가]
C1775               [영구임대, 임대상가]
C1790               [영구임대, 임대상가]
C1859               [영구임대, 임대상가]
C1874               [영구임대, 임대상가]
C1875               [영구임대, 임대상가]
C1899               [영구임대, 임대상가]
C1925               [영구임대, 임대상가]
C1983    [공공임대(50년), 영구임대, 임대상가]
C2034               [영구임대, 임대상가]
C2038               [영구임대, 임대상가]
C2109               [영구임대, 임대상가]
C2132               [영구임대, 임대상가]
C2135               [영구임대, 임대상가]
C2190               [영구임대, 임대상가]
C2212               [영구임대, 임대상가]
C2258               [영구임대, 임대상가]
C2289               [영구임대, 임대상가]
C2310               [영구임대, 임대상가]
C2416               [영구임대, 임대상가]
C2571               [영구임대, 임대상가]
C2597

In [16]:
# 임대상가 갯수 변수 만들기
temp = 임대상가아파트.groupby(임대상가아파트.단지코드)
단지별임대 = temp.공급유형.value_counts()[:,'임대상가']
단지별임대 = 단지별임대.reset_index('단지코드')
단지별임대.rename(columns = {"공급유형" : "rent_mall"}, inplace = True)
단지별임대.head(3)


,단지코드,rent_mall
0,C1004,17
1,C1056,10
2,C1109,20


In [17]:
train3 = pd.merge(train2, 단지별임대, on='단지코드',how='left') #임대상가가 있으면 단지별임대값을 넣음
train3.rent_mall[train3.rent_mall.isnull()] = 0 #임대상가가 없으면 0 넣음.
train3.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수,ssd,dif,rent_mall
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17,A,9216000,82940,0.0,3,624,205,545,0,0.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17,A,12672000,107130,0.0,3,624,205,545,0,0.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17,A,12672000,107130,0.0,3,624,205,545,0,0.0


In [18]:
# test 적용
# 임대가 같이 있는지?
임대상가 = test2.loc[test2.공급유형 == "임대상가"]
임대단지코드 = 임대상가.단지코드.unique()
임대상가아파트 = test2.loc[test2.단지코드.isin(임대단지코드)]
임대상가아파트.head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,ssd,dif
73,C1006,1505,아파트,대전광역시,영구임대,26.37,358,27,C,5787000,79980,2.0,5,428,1505,0
74,C1006,1505,아파트,대전광역시,영구임대,26.37,229,27,C,5787000,79980,2.0,5,428,1505,0
75,C1006,1505,아파트,대전광역시,영구임대,26.37,574,27,C,5787000,79980,2.0,5,428,1505,0


In [19]:
# 임대상가가 있는 아파트 유형 확인
임대상가아파트.groupby(임대상가아파트.단지코드).공급유형.unique() # 7개 단지코드

단지코드
C1006    [영구임대, 임대상가]
C1729    [영구임대, 임대상가]
C1812    [영구임대, 임대상가]
C2033    [영구임대, 임대상가]
C2177    [영구임대, 임대상가]
C2253    [영구임대, 임대상가]
C2676    [영구임대, 임대상가]
Name: 공급유형, dtype: object

In [20]:
# 임대상가 갯수 변수 만들기
temp = 임대상가아파트.groupby(임대상가아파트.단지코드)
단지별임대 = temp.공급유형.value_counts()[:,'임대상가']
단지별임대 = 단지별임대.reset_index('단지코드')
단지별임대.rename(columns = {"공급유형" : "rent_mall"}, inplace = True)
단지별임대.head(3)

,단지코드,rent_mall
0,C1006,20
1,C1729,17
2,C1812,25


In [21]:
test3 = pd.merge(test2, 단지별임대, on='단지코드',how='left') #임대상가가 있으면 단지별임대값을 넣음
test3.rent_mall[test3.rent_mall.isnull()] = 0 #임대상가가 없으면 0 넣음.
test3.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,ssd,dif,rent_mall
0,C1072,754,아파트,경기도,국민임대,39.79,116,14,H,22830000,189840,0.0,2,683,754,0,0.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14,A,36048000,249930,0.0,2,683,754,0,0.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14,H,36048000,249930,0.0,2,683,754,0,0.0


In [22]:
# 임대료 빈 곳 넣기
# train
# factor형으로 값 추가할 수 없어서, character형으로 변환
train3.loc[(train3.단지코드 == "C1326") & (train3.전용면적 == 24.72),['임대보증금','임대료']] = [7287000,140530]
train3.loc[(train3.단지코드 == "C1326") & (train3.전용면적 == 24.79),['임대보증금','임대료']] = [7287000,140530]
train3.loc[(train3.단지코드 == "C1326") & (train3.전용면적 == 46.94),['임대보증금','임대료']] = [24047000,269610]
train3.loc[(train3.단지코드 == "C1326") & (train3.전용면적 == 26.83),['임대보증금','임대료']] = [911000,147820]
train3.loc[(train3.단지코드 == "C1326") & (train3.전용면적 == 37.7),['임대보증금','임대료']] = [15406000,206110]

train3.loc[(train3.단지코드 == "C1786") & (train3.전용면적 == 16.91),['임대보증금','임대료']] = [13450000,65500]
train3.loc[(train3.단지코드 == "C1786") & (train3.전용면적 == 26.9),['임대보증금']] = np.repeat(19820000,3)
train3.loc[(train3.단지코드 == "C1786") & (train3.전용면적 == 26.9),['임대료']] = np.repeat(97000,3)

train3.loc[(train3.단지코드 == "C2186") & (train3.전용면적 == 29.17),['임대보증금','임대료']] = [9692000,147880]
train3.loc[(train3.단지코드 == "C2186") & (train3.전용면적 == 29.34),['임대보증금','임대료']] = [9692000,147880]
train3.loc[(train3.단지코드 == "C2186") & (train3.전용면적 == 37.43),['임대보증금','임대료']] = [18319000,210240]



In [23]:
train3.임대료 = train3.임대료.replace('-', np.NaN) # '-'를 NaN으로 변경해야 float형으로 변경 가능

In [24]:
# 형변환
train3 = train3.astype({'임대보증금' : np.float,
                        '임대료' : np.float})

In [25]:
# 결측값 있는지 double check
print(pd.isnull(train3.임대보증금).sum())
print(pd.isnull(train3.임대료).sum())
### NULL값 존재함

569
572


In [26]:
# test
test3.loc[(test3.단지코드 == "C1267") & (test3.전용면적 == 16.94),['임대보증금','임대료']] = [11200000,53200]
test3.loc[(test3.단지코드 == "C1267") & (test3.전용면적 == 26.85),['임대보증금']] = [16333333,18620000]
test3.loc[(test3.단지코드 == "C1267") & (test3.전용면적 == 26.85),['임대료']] = [77580,88440]
test3.loc[(test3.단지코드 == "C1267") & (test3.전용면적 == 36.77),['임대보증금','임대료']] = [23760000,112860]

test3.loc[(test3.단지코드 == "C1006") & (test3.전용면적 == 52.74),['임대보증금','임대료']] = [5000000,99520]

test3.loc[(test3.단지코드 == "C2152") & (test3.전용면적 == 24.83),['임대보증금','임대료']] = [2129000,42350]
test3.loc[(test3.단지코드 == "C2152") & (test3.전용면적 == 33.84),['임대보증금','임대료']] = [2902000,57730]



In [27]:
# 형변환
test3 = test3.astype({'임대보증금' : np.float,
                        '임대료' : np.float})

In [28]:
# 결측값 있는지 double check
print(pd.isnull(test3.임대보증금).sum())
print(pd.isnull(test3.임대료).sum())
### NULL값 존재함

179
179


In [29]:
#임대 상가, 장기전세, 공공분양 row 빼기
# train
삭제 = ['장기전세', '임대상가','공공분양']
train3_1 = train3.loc[-train3.공급유형.isin(삭제)]
# 제대로 제거되었는지 확인
train3_1.공급유형.unique()

array(['국민임대', '공공임대(50년)', '영구임대', '공공임대(10년)', '공공임대(분납)', '행복주택',
       '공공임대(5년)'], dtype=object)

In [30]:
# test
삭제 = ['장기전세', '임대상가','공공분양']
test3_1 = test3.loc[-test3.공급유형.isin(삭제)]
# 제대로 제거되었는지 확인
test3_1.공급유형.unique()

array(['국민임대', '영구임대', '공공임대(50년)', '공공임대(10년)', '공공임대(분납)', '행복주택'],
      dtype=object)

In [31]:
train3_1 = train3_1.drop('dif', axis=1)
test3_1 = test3_1.drop('dif', axis=1)

In [32]:
real_train = pd.read_csv('train3_1.csv',encoding='cp949')
real_test = pd.read_csv('test3_1.csv',encoding='cp949')
real_train.rename(columns = {'도보.10분거리.내.지하철역.수.환승노선.수.반영.' : '지하철',"도보.10분거리.내.버스정류장.수" : "버스"}, inplace = True)
real_test.rename(columns = {'도보.10분거리.내.지하철역.수.환승노선.수.반영.' : '지하철',"도보.10분거리.내.버스정류장.수" : "버스"}, inplace = True)

real_test.tail(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,ssd,rent_mall
828,C2189,382,아파트,전라북도,국민임대,29.19,20,45,H,6872000.0,106400.0,0,2,300,378,0
829,C2189,382,아파트,전라북도,국민임대,39.45,202,45,H,13410000.0,144600.0,0,2,300,378,0
830,C2189,382,아파트,전라북도,국민임대,46.23,60,45,H,18689000.0,166500.0,0,2,300,378,0


In [33]:
# test3_1.to_csv('py_test3.csv',header=True, index=False, encoding='utf-8-sig')
# train3_1.to_csv('py_train3.csv',header=True, index=False, encoding='utf-8-sig')

In [34]:
print(pd.isnull(test3_1.임대보증금).sum())
print(pd.isnull(real_test.임대보증금).sum())
print(pd.isnull(train3_1.임대보증금).sum())
print(pd.isnull(real_train.임대보증금).sum())

print(pd.isnull(test3_1.임대료).sum())
print(pd.isnull(real_test.임대료).sum())
print(pd.isnull(train3_1.임대료).sum())
print(pd.isnull(real_train.임대료).sum())

2
2
0
0
2
2
0
0


In [35]:
# 파이썬으로 만든 데이터셋과 소담쓰 데이터셋이 동일한지 비교 (결과로 아무것도 안 나오면 동일한 것임.)
df = pd.concat([real_train,train3_1])
df = df.reset_index(drop=True)
df_gpby = df.groupby(list(df.columns)) # 그룹화
idx = [x[0] for x in df_gpby.groups.values() if len(x) == 1] # 고유 레코드 색인을 얻음
df.reindex(idx) #필터

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수,ssd,rent_mall


In [ ]:
# 파이썬으로 만든 데이터셋과 소담쓰 데이터셋이 동일한지 비교 (결과로 아무것도 안 나오면 동일한 것임.)
# 동일해보이는데... 뭐가 문제일까...ㅎ
df = pd.concat([test3_1,real_test])
df = df.reset_index(drop=True)
df_gpby = df.groupby(list(df.columns)) # 그룹화
idx = [x[0] for x in df_gpby.groups.values() if len(x) == 1] # 고유 레코드 색인을 얻음
df.reindex(idx) #필터

* train_code_u : 공급유형별, 단지코드별 group by 한 데이터
(가령 C1000에 국민임대와 영구임대가 있을경우,
C1000-1 & 국민임대
C1000-2 & 영구임대 
로 나눔)
* train_code : 단지코드별로 group by 한 데이터
(앞선 C1000의 경우 공급유형에 상관없이 전체 임대료/세대수 등의 평균값으로 들어가고, 국민임대/영구임대 로 함께 작성되는 새 범주 생성)

* train_code_no: 공급유형을 삭제하고 단지코드별로 group by 한 데이터

In [62]:
train_code_u = pd.read_csv("train_code_u.csv", encoding="cp949")
test_code_u = pd.read_csv("test_code_u.csv", encoding="cp949")

In [44]:
train_code = pd.read_csv("train_code.csv", encoding="cp949")
test_code = pd.read_csv("test_code.csv", encoding="cp949")

In [47]:
train_code_no = pd.read_csv("train_code_no.csv", encoding="cp949")
test_code_no = pd.read_csv("test_code_no.csv", encoding="cp949")

Catboost 모델링

In [ ]:
# !pip install catboost

In [48]:
from catboost import CatBoostRegressor


In [50]:
cat_features = ['공급유형','gubun','area']  #범주형 변수
target_col = 'Y' # Y 변수

catboost 모델 적합  
https://2econsulting.github.io/catboost/2018/06/16/catboostSimpleVersion.html  
특히 CatBoost의 full name은 Categorical Boost으로 범주형 변수가 많은 데이터셋에서 예측 성능이 우수

In [59]:
cat_model = CatBoostRegressor(
         cat_features=cat_features,
         loss_function='MAE',
         n_estimators=500, 
         learning_rate=0.1, 
         random_state=3978
    )
    
cat_model.fit(train_code_u.drop(columns=[target_col,'NEW_ID']), train_code_u[target_col], 
              verbose=100)

0:	learn: 268.4713623	total: 4.12ms	remaining: 2.05s
100:	learn: 63.9736049	total: 361ms	remaining: 1.42s
200:	learn: 42.8627857	total: 699ms	remaining: 1.04s
300:	learn: 33.7069623	total: 1.05s	remaining: 696ms
400:	learn: 28.1615521	total: 1.4s	remaining: 344ms
499:	learn: 25.1079565	total: 1.75s	remaining: 0us


In [40]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission.head()

,code,num
0,C1072,0
1,C1128,0
2,C1456,0
3,C1840,0
4,C1332,0


In [69]:
## catboost모델 예측값
result1 = cat_model.predict(test_code_u.drop(columns=['NEW_ID']))

In [80]:
cat_result1 = pd.DataFrame(test_code_u.loc[:,'NEW_ID'])
cat_result1['num'] = result1
cat_result1.head(3)

,NEW_ID,num
0,C1003_1,254.153142
1,C1006_1,342.310001
2,C1016_1,670.763803


In [89]:
## 여러행의 정보가 담겨져 있는 단지코드 정보 추출
def sub_code(x):
   return x[:5]   # NEW_ID에서 단지코드 5글자만 추출

단지코드만 = cat_result1.NEW_ID.apply(sub_code)
여러단지코드 = 단지코드만[단지코드만.duplicated()].unique()
여러단지코드
cat_result1['code'] = 단지코드만

array(['C1147', 'C1267', 'C1849', 'C2091', 'C2113', 'C2131', 'C2430',
       'C2688'], dtype=object)

In [ ]:
## 여러행의 정보가 담겨져 있는 단지코드의 결과는 MEAN값으로 넣어줌.
temp = cat_result1.loc[단지코드만.isin(여러단지코드)]
temp['code'] = 단지코드만
temp2 = temp.num.groupby(temp.code).mean()
temp3 = pd.DataFrame(temp2).reset_index()

In [163]:
## cat_boost의 결과값 + 위에서 만든 평균값(temp3) = 147개이므로
## submission에 147개의 결과를 left_join 시켜서 최종 제출결과값 cat_result5를 만듦.
cat_result2 = cat_result1[['code','num']]
cat_result3 = cat_result2.loc[-cat_result2.code.isin(여러단지코드)]
cat_result4 = pd.concat([cat_result3, temp3])
cat_result5 = pd.merge(submission, cat_result4, on='code',how='left')
cat_result5.drop('num_x',axis=1, inplace=True)
cat_result5.rename(columns={'num_y':'num'}, inplace=True)
cat_result5.head(3)

,code,num
0,C1072,813.540274
1,C1128,1241.575843
2,C1456,521.333601


In [151]:
cat_result5.to_csv('cat_result1.csv',index=False,encoding='cp949')